In [ ]:
# This program uses LSTM to predict closing stock price of a coperation using last 60 day stock price


In [ ]:
import math
import pandas_datareader as web 
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense ,LSTM 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Getting the Stock 
df=web.DataReader('AAPL',data_source='yahoo' ,start='2015-01-01',end='2021-04-01')
#Show data
df

In [ ]:
df.shape

In [ ]:
#Visualization of Closing Price
plt.figure(figsize=(16,8))
plt.title('Closing Price History')
plt.plot(df['Close'])
plt.xlabel('Data',fontsize=18)
plt.ylabel('Close Price USD($)',fontsize=18)
plt.show

In [ ]:
#New Dataframe with only Close
data= df.filter(['Close'])
#Convert dataset into numpy array
dataset = data.values
#Get the number of Rows to Train the Dataset
training_data_len= math.ceil(len(dataset)*0.8)

training_data_len

In [ ]:
#Scale the Data
scaler= MinMaxScaler(feature_range=(0,1))
scaled_data= scaler.fit_transform(dataset)

scaled_data

In [ ]:
#Create the traing dataset 
#Create the scaled training dataset
train_data=scaled_data[0:training_data_len , :]

#Split the data into x_train and y_train
x_train=[]
y_train=[]
 
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])
  if i<=60:
    print(x_train)
    print(y_train)
    print() 

In [ ]:
#convert X-train and Y_train into nupy array
x_train,y_train = np.array(x_train),np.array(y_train)

#Reshape the Data
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
#Build LSTM Model
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False ))
model.add(Dense(25))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')



In [ ]:
model.summary()

In [ ]:
#Train the Dataset
model.fit(x_train,y_train,batch_size=1,epochs=1)

In [ ]:
#Create the Testing Dataset
test_data = scaled_data[training_data_len - 60: , :]

#Create test dataset x_test and y_test
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])



In [ ]:
#Convert data into numpy array  
x_test = np.array(x_test)

In [ ]:
#Reshape the data
x_test = np.reshape(x_test , (x_test.shape[0], x_test.shape[1], 1) )

In [ ]:
#Get Model to Prediction Price Value

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)



In [ ]:
#Getting the Root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)** 2)
rmse

3.893642000332

In [ ]:
#Plot the Data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Visualize the Data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD($)' ,fontsize =18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'],loc = 'lower right')
plt.show()



In [ ]:
#Show Valid and Predicted Prices
valid

In [ ]:
#Get the Quote
apple_quote= web.DataReader('AAPL',data_source='yahoo' ,start='2015-01-01',end='2021-04-30')
#Create new Dataframe
new_df= apple_quote.filter(['Close'])
#Get last 60 days Closing value and get data frame into array
last_60_days= new_df[-60:].values
#Scale taht data B/w 0 & 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append last 60 days
X_test.append(last_60_days_scaled)
#Convert X_test into numpy
X_test= np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test , (X_test.shape[0], X_test.shape[1], 1) )
#Get the predicted scale value
pred_price=model.predict(X_test)
#undo the Scaling
pred_price = scaler.inverse_transform(pred_price)

print(pred_price)

[[140.6808]]


In [ ]:
#get the quote
apple_quote2= web.DataReader('AAPL',data_source='yahoo' ,start='2021-04-29',end='2021-04-30')
print(apple_quote2['Close'])
